# Energy savings on multicell bayesian digital twins
## Demonstrate how radp helps to map the pixel with the best cell on the site

# Prerequisite

### Sample data set

Unpack the sample data set present at `notebooks/data/sim_data.zip` under `notebooks/data/`


In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

#### Completely reproducible results are not guaranteed across PyTorch releases, individual commits, or different platforms. Furthermore, results may not be reproducible between CPU and GPU executions, even when using identical seeds.

Hence will test results using same bayesian digitaltwins but 2 different EnergySavingsGym object and match the results

In [ ]:
from radp.utility.simulation_utils import seed_everything
seed_everything(1)

In [ ]:
import numpy as np
from scipy import stats
from scipy.ndimage import correlate
from skimage.metrics import structural_similarity as ssim
from radp_library import *

## Using pregenerated data stored locally
Currently the data is stored under notebooks

/data folder

In [ ]:
WORKING_DIR = f"{Path().absolute()}"
BUCKET_PATH = f"{WORKING_DIR}/data"
SIM_DATA_PATH = "sim_data/3cell"

## Bayesian digital twin training

In [ ]:
# provide list of folder name under which the pregenerated data is stored
sim_idx_folders = ['sim_001', 'sim_002', 'sim_003', 'sim_004', 'sim_005']

In [ ]:
p_train_maxiter_dict = {
        40: [5]
}
p_test = 100

bayesian_digital_twins_list = []
test_data_list = []
pred_rsrp_list = []
MAE_list = []
Percentile85Error_list = []
p_train_list = []
maxiter_list = []


for p_train in p_train_maxiter_dict.keys():
    for maxiter in p_train_maxiter_dict[p_train]:
        logging.info(f"\n\nMAXITER = {maxiter}, p_train={p_train}\n")
        bayesian_digital_twins, site_config_df, test_data, loss_vs_iter, lons, lats, true_rsrp, pred_rsrp, MAE, Percentile85Error = bdt(
            bucket_path=BUCKET_PATH,
            sim_data_path=SIM_DATA_PATH,
            p_train=p_train,
            p_test=p_test,
            maxiter=maxiter,
            sim_idx_folders=sim_idx_folders,
            test_idx=2,
            plot_loss_vs_iter=True,
            choose_strongest_samples_percell=False,
            filter_out_samples_dbm_threshold=-70,
            filter_out_samples_kms_threshold=0.65,
        )
        bayesian_digital_twins_list.append(bayesian_digital_twins)
        test_data_list.append(test_data)
        p_train_list.append(p_train)
        maxiter_list.append(maxiter)
        pred_rsrp_list.append(pred_rsrp)
        MAE_list.append(MAE)
        Percentile85Error_list.append(Percentile85Error)


## Construct EnergySavings OpenAI Gym object

In [ ]:
from apps.energy_savings.energy_savings_gym import EnergySavingsGym
energy_savings_gym = EnergySavingsGym(
    bayesian_digital_twins=bayesian_digital_twins_list[0],
    site_config_df=site_config_df[site_config_df.cell_id.isin(bayesian_digital_twins_list[0].keys())].reset_index(),
    prediction_frame_template=test_data_list[0],
    tilt_set=[0, 1],
)

## Run a few iterations of Energy Savings OpenAI gym steps

In [ ]:
import time
iterations = 100

In [ ]:
start = time.time()
expected_rewards = []
for _i in range(iterations):
    # Sample a random action from the entire action space
    random_action = energy_savings_gym.action_space.sample()
    # Take the action and get the new observation
    new_obs, reward, done, info = energy_savings_gym.step(random_action)
    expected_rewards.append(reward)
    print(reward)
end = time.time()
print(f"Finished {iterations} iterations in {(end - start)} seconds!")

## Reproducbility test
Generate reproducible results using identical seeds which was set at the begining of the notebook using seed_everything(1)

In [ ]:
energy_savings_gym = EnergySavingsGym(
    bayesian_digital_twins=bayesian_digital_twins_list[0],
    site_config_df=site_config_df[site_config_df.cell_id.isin(bayesian_digital_twins_list[0].keys())].reset_index(),
    prediction_frame_template=test_data_list[0],
    tilt_set=[0, 1],
)

In [ ]:
start = time.time()
rewards = []

for _i in range(iterations):
    # Sample a random action from the entire action space
    random_action = energy_savings_gym.action_space.sample()
    # Take the action and get the new observation
    new_obs, reward, done, info = energy_savings_gym.step(random_action)
    rewards.append(reward)
    print(reward)
end = time.time()
print(f"Finished {iterations} iterations in {(end - start)} seconds!")

In [ ]:
print(rewards==expected_rewards)